In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from matplotlib.ticker import MaxNLocator
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split
import os
import time
import numpy as np
import pandas as pd
from sklearn.metrics import log_loss, accuracy_score
from sklearn.model_selection import train_test_split
import torch
from lightautoml.automl.presets.tabular_presets import TabularAutoML, TabularUtilizedAutoML
from lightautoml.tasks import Task
from lightautoml.report.report_deco import ReportDeco
import seaborn as sns
import lightgbm as ltb
from xgboost import XGBRegressor
!pip install -U lightautoml

In [ ]:
df=pd.read_csv("../input/tabular-playground-series-mar-2022/train.csv",parse_dates=True)
test=pd.read_csv("../input/tabular-playground-series-mar-2022/test.csv")
sub=pd.read_csv("../input/tabular-playground-series-mar-2022/sample_submission.csv")


In [ ]:
df.time=df.time.astype('str')
test.time=df.time.astype('str')

# Reduce Memory usage

In [ ]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)  
        else:
            df[col] = df[col].astype('category')
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df
reduce_mem_usage(df)
reduce_mem_usage(test)

# Converting time column to Date_time

In [ ]:
df['time'] = pd.to_datetime(df['time'])
test['time'] = pd.to_datetime(test['time'])
pt_df=df.copy()
pt_test=test.copy

# Feature Engineering

In [ ]:
def features_new(data):
    data['month'] = data['time'].dt.month
    data['weekday'] = data['time'].dt.weekday
    data['hour'] = data['time'].dt.hour
    data['minute'] = data['time'].dt.minute
    data['is_month_start'] = data['time'].dt.is_month_start.astype('int')
    data['is_month_end'] = data['time'].dt.is_month_end.astype('int')
    data['hour+minute'] = data['time'].dt.hour * 60 + data['time'].dt.minute
    data['is_weekend'] = (data['time'].dt.dayofweek > 4).astype('int')
    data['is_afternoon'] = (data['time'].dt.hour > 12).astype('int')
    data['x+y'] = data['x'].astype('str') + data['y'].astype('str')
    data['x+y+direction'] = data['x'].astype('str') + data['y'].astype('str') + data['direction'].astype('str')
    data['hour+direction'] = data['hour'].astype('str') + data['direction'].astype('str')
    data['hour+x+y'] = data['hour'].astype('str') + data['x'].astype('str') + data['y'].astype('str')
    data['hour+direction+x'] = data['hour'].astype('str') + data['direction'].astype('str') + data['x'].astype('str')
    data['hour+direction+y'] = data['hour'].astype('str') + data['direction'].astype('str') + data['y'].astype('str')
    data['hour+direction+x+y'] = data['hour'].astype('str') + data['direction'].astype('str') + data['x'].astype('str') + data['y'].astype('str')
    data['hour+x'] = data['hour'].astype('str') + data['x'].astype('str')
    data['hour+y'] = data['hour'].astype('str') + data['y'].astype('str')
    data.direction=data.direction.astype(str)
features_new(df)
features_new(test)

# Adding weekend ⛪🏡 *Saturday* and *Sunday* as features

In [ ]:

df['sunday'] = np.where(df['weekday']==6,1, 0)
df['saturday'] = np.where(df['weekday']==5,1, 0)

test['sunday'] = np.where(test['weekday']==6,1, 0)
test['saturday'] = np.where(test['weekday']==5,1, 0)


# Plotting Bar chart for congestion🚗🚙🚔
* It can be observed from plot that some values have higher congestion than their neighbouring values.
    * Values with traffic **🚗🚗🚙** **congestion=15,20,21,29,34** has much higher count than its neighbours.    

In [ ]:
plt.figure(figsize=(20, 6))
plt.bar(range(101), pt_df.congestion.value_counts().sort_index(), width=1,
        color=['gold' if con in [15, 20, 21, 29, 34] else 'coral' for con in range(101)])
plt.title("Congestion count plot")
plt.xlabel("Congestion")

# Dependence of congestion 🚗🚙 on time of day
* Maximum traffic congestion is at 5:00 P.M
* Minimum traffic congestion is at 4:00 A.M

In [ ]:

plt.figure(figsize=(18, 6))
plt.bar(pt_df.groupby(pt_df.time.dt.hour).congestion.mean().index,pt_df.groupby(df.time.dt.hour).congestion.mean(),color="coral")
plt.xticks(range(24))
plt.ylabel("congestion")
plt.title("Congestion on time")
plt.show()

# Traffic 🚗🚙 Congestion on Date

In [ ]:
plt.figure(figsize=(18, 6))
plt.bar(df.groupby(pt_df.time.dt.date).congestion.mean().index,pt_df.groupby(pt_df.time.dt.date).congestion.mean(),color="coral")
plt.ylabel("congestion")
plt.title("Congestion on date")
plt.ylim(40, 52)
plt.show()

# Traffic 🚗🚙 Congestion on day of week
* It is observed that congestion is less on weekend.

In [ ]:
plt.figure(figsize=(18, 6))
plt.bar(pt_df.groupby(pt_df.time.dt.dayofweek).congestion.mean().index,pt_df.groupby(pt_df.time.dt.dayofweek).congestion.mean(),color="coral")
plt.xticks(range(7))
plt.ylabel("congestion")
plt.title("Congestion on day of week")
plt.show()

In [ ]:
# from sklearn import preprocessing
# cat=[col for col in df.columns if df[col].dtype=="object"]
# encoder = preprocessing.LabelEncoder()
# for i in cat:
#     encoder = preprocessing.LabelEncoder()
#     df[i] = encoder.fit_transform(df[i])
#     test[i] = encoder.transform(test[i])
#     print(i,"==> unique",df[i].nunique())

In [ ]:
# from sklearn.preprocessing import OneHotEncoder
# cat=[col for col in df.columns if df[col].dtype=="object"]
# OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
# train = pd.DataFrame(OH_encoder.fit_transform(df[cat]),columns=list(OH_encoder.get_feature_names()))
# en_test = pd.DataFrame(OH_encoder.transform(test[cat]),columns=list(OH_encoder.get_feature_names()))
# # One-hot encoding removed index; put it back
# train.index = df.index
# en_test.index = test.index
# df=df.drop(cat,axis=1)
# test=test.drop(cat,axis=1)
# df=pd.concat([df,train],axis=1)
# test=pd.concat([test,en_test],axis=1)

# Useful Feats

In [ ]:
feats=[col for col in df.columns if col not in ["row_id","congestion","time"]]

# Light-AUTO ML

In [ ]:
import time
np.random.seed(2022)
torch.set_num_threads(4)
TARGET_NAME = 'congestion'
task = Task('reg', metric='mae', loss='mae')
roles = {'target': TARGET_NAME,
         'drop': ['row_id']
         }
%time 
automl = TabularAutoML(task = task,
                       timeout = 8*3600,
                       cpu_limit = 4,
                       reader_params = {'n_jobs': 4, 'random_state': 2022},
                       general_params = {'use_algos': [['lgb']]}
                      )

In [ ]:
oof_pred = automl.fit_predict(df, roles = roles, verbose=1)

# Feature Importance

In [ ]:
%%time

fast_fi = automl.get_feature_scores('fast')
fast_fi.set_index('Feature')['Importance'].plot.bar(figsize=(20, 10), grid=True)

# Predictions

In [ ]:
%%time
test_pred = automl.predict(test)

# Submission

In [ ]:
sub[TARGET_NAME] = test_pred.data[:, 0].round()
sub.to_csv('submission.csv', index=False)
sub